# 🌍 Responsible Airbnb Recommendations - Suggest Less Busy Alternatives
This notebook allows users to explore NYC Airbnb neighbourhoods and, **only if a busy area is selected**, recommends nearby alternatives that are **less busy** and close by.

## 📦 Dependencies

In [ ]:
# !pip3 install pandas geopandas shapely scikit-learn matplotlib ipywidgets

## 🔧 Import Libraries

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

## 📊 Load and Prepare Dataset

In [7]:
df = pd.read_csv("../data/AB_NYC_2019.csv")
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
df['busyness_score'] = df['reviews_per_month'] * df['availability_365']

agg_geo = df.groupby(['neighbourhood_group', 'neighbourhood']).agg({
    'latitude': 'mean',
    'longitude': 'mean',
    'price': 'mean',
    'availability_365': 'mean',
    'reviews_per_month': 'mean',
    'busyness_score': 'mean'
}).reset_index()

# Define threshold for busyness
busy_threshold = df['busyness_score'].quantile(0.75)

# Separate busy and non-busy
busy_geo = agg_geo[agg_geo['busyness_score'] >= busy_threshold].copy()
quiet_geo = agg_geo[agg_geo['busyness_score'] < busy_threshold].copy()

# Create GeoDataFrames
geometry_busy = [Point(xy) for xy in zip(busy_geo['longitude'], busy_geo['latitude'])]
geometry_quiet = [Point(xy) for xy in zip(quiet_geo['longitude'], quiet_geo['latitude'])]

gdf_busy = gpd.GeoDataFrame(busy_geo, geometry=geometry_busy, crs="EPSG:4326")
gdf_quiet = gpd.GeoDataFrame(quiet_geo, geometry=geometry_quiet, crs="EPSG:4326")

# Fit KNN only on less busy options
quiet_coords = gdf_quiet[['latitude', 'longitude']]
nn_quiet_model = NearestNeighbors(n_neighbors=5, metric='haversine')
nn_quiet_model.fit(np.radians(quiet_coords))

NearestNeighbors(metric='haversine')

## 🎛️ Select a Busy Neighbourhood

In [12]:
group_dropdown = widgets.Dropdown(
    options=gdf_busy['neighbourhood_group'].unique(),
    description='Borough:',
    style={'description_width': 'initial'}
)

neigh_dropdown = widgets.Dropdown(
    options=[],
    description='Neighbourhood:',
    style={'description_width': 'initial'}
)

output = widgets.Output()

def update_dropdowns(*args):
    group = group_dropdown.value
    neigh_dropdown.options = gdf_busy[gdf_busy['neighbourhood_group'] == group]['neighbourhood'].unique()

group_dropdown.observe(update_dropdowns, 'value')
update_dropdowns()

## 📍 Suggest Less Busy Alternatives if Needed

In [16]:
def suggest_alternatives(b):
    output.clear_output()
    selected_group = group_dropdown.value
    selected_neigh = neigh_dropdown.value

    selected_point = gdf_busy[
        (gdf_busy['neighbourhood_group'] == selected_group) &
        (gdf_busy['neighbourhood'] == selected_neigh)
    ][['latitude', 'longitude']].values[0]

    distances, indices = nn_quiet_model.kneighbors([np.radians(selected_point)])
    suggestions = gdf_quiet.iloc[indices[0]].copy()
    suggestions['distance_km'] = distances[0] * 6371

    selected_busyness = gdf_busy[
        (gdf_busy['neighbourhood_group'] == selected_group) &
        (gdf_busy['neighbourhood'] == selected_neigh)
    ]['busyness_score'].values[0]

    with output:
        print(f"Busyness score for '{selected_neigh}': {selected_busyness:.2f}\\n")
        print(f"Suggestions for less busy neighbourhoods near '{selected_neigh}':")
        display(suggestions[['neighbourhood_group', 'neighbourhood', 'distance_km', 'busyness_score']])

        fig, ax = plt.subplots(figsize=(10, 6))
        base = gdf_quiet.plot(ax=ax, color='lightgrey', markersize=5)
        suggestions.plot(ax=base, color='green', markersize=50, label='Less Busy Alternatives')
        gdf_busy[gdf_busy['neighbourhood'] == selected_neigh].plot(ax=base, color='red', markersize=100, label='Selected Area')
        plt.legend()
        plt.title('Suggested Neighbourhoods')
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()


button = widgets.Button(description="Suggest Alternatives")
button.on_click(suggest_alternatives)

display(widgets.VBox([group_dropdown, neigh_dropdown, button, output]))